In [2]:
import os
import requests
import boto3
from botocore.exceptions import NoCredentialsError

# Initialize S3 client
s3_client = boto3.client('s3')

def upload_to_s3(file_path, bucket, folder, object_name=None):
    """Upload a file to an S3 bucket

    :param file_path: File to upload
    :param bucket: Bucket to upload to
    :param folder: Folder in the bucket to upload to
    :param object_name: S3 object name. If not specified, then file_path is used
    :return: True if file was uploaded, else False
    """
    if object_name is None:
        object_name = os.path.basename(file_path)

    s3_key = f"{folder}/{object_name}"

    try:
        s3_client.upload_file(file_path, bucket, s3_key)
        print(f"File {file_path} uploaded to {bucket}/{s3_key}")
        return True
    except NoCredentialsError:
        print("Credentials not available")
        return False

def get_tif_url(target_year):
    """Get the TIF URL for the specified year and crop

    :param target_year: Year to get the data for
    :return: TIF URL
    """
    BBOX_LL_X = -2274195
    BBOX_LL_Y = 1785375
    BBOX_UR_X = -2196105
    BBOX_UR_Y = 1879155
    TARGET_CROP = 221

    # Get CDL file URL
    cdl_url = requests.get(
        f"https://nassgeodata.gmu.edu/axis2/services/CDLService/GetCDLFile?year={target_year}&bbox={BBOX_LL_X},{BBOX_LL_Y},{BBOX_UR_X},{BBOX_UR_Y}"
    ).text

    cdl_url = cdl_url.split("<returnURL>")[1].split("</returnURL>")[0]

    # Get TIF file URL
    tif_url = requests.get(
        f"https://nassgeodata.gmu.edu/axis2/services/CDLService/ExtractCDLByValues?file={cdl_url}&values={TARGET_CROP}"
    ).text

    tif_url = tif_url.split("<returnURL>")[1].split("</returnURL>")[0]

    return tif_url


def get_tif_url_by_fips(target_year,fips_code):
    """Get the TIF URL for the specified year, fips code, and crop

    :param target_year: Year to get the data for
    :param fips_code: county code of target area
    :return: TIF URL
    """

    TARGET_CROP = 221

    # Get CDL file URL
    cdl_url = requests.get(
        f"https://nassgeodata.gmu.edu/axis2/services/CDLService/GetCDLFile?year={target_year}&fips={fips_code}"
    ).text

    cdl_url = cdl_url.split("<returnURL>")[1].split("</returnURL>")[0]

    # Get TIF file URL
    tif_url = requests.get(
        f"https://nassgeodata.gmu.edu/axis2/services/CDLService/ExtractCDLByValues?file={cdl_url}&values={TARGET_CROP}"
    ).text

    tif_url = tif_url.split("<returnURL>")[1].split("</returnURL>")[0]

    return tif_url

def process_year(target_year):
    print(f"Pulling Strawberry Data for year {target_year}")

    # Get the TIF URL
    tif_url = get_tif_url(target_year)

    # Download the file
    local_file = f"cropscape_strawberries_{target_year}.tif"
    response = requests.get(tif_url)

    with open(local_file, 'wb') as f:
        f.write(response.content)

    print(f"File downloaded to {local_file}")

    # Upload to S3
    bucket_name = 'agrisense'
    folder_name = 'cropscape'
    upload_to_s3(local_file, bucket_name, folder_name)

    # Optionally, clean up the local file
    os.remove(local_file)
    print(f"Local file {local_file} deleted.")

def process_year_fips(target_year,fips_code):
    print(f"Pulling Strawberry Data for year {target_year} and {fips_code} FIPS")

    # Get the TIF URL
    tif_url = get_tif_url_by_fips(target_year,fips_code)

    # Download the file
    local_file = f"cropscape-strawberries-{fips_code}-{target_year}.tif"
    response = requests.get(tif_url)

    with open(local_file, 'wb') as f:
        f.write(response.content)

    print(f"File downloaded to {local_file}")

    # Upload to S3
    bucket_name = 'agrisense'
    folder_name = 'cropscape_monterey'
    upload_to_s3(local_file, bucket_name, folder_name)

    # Optionally, clean up the local file
    os.remove(local_file)
    print(f"Local file {local_file} deleted.")

def main():
    for year in range(2010, 2024):
        process_year_fips(year,'06053')

if __name__ == "__main__":
    main()


Pulling Strawberry Data for year 2010 and 06053 FIPS
File downloaded to cropscape-strawberries-06053-2010.tif
File cropscape-strawberries-06053-2010.tif uploaded to agrisense/cropscape_monterey/cropscape-strawberries-06053-2010.tif
Local file cropscape-strawberries-06053-2010.tif deleted.
Pulling Strawberry Data for year 2011 and 06053 FIPS
File downloaded to cropscape-strawberries-06053-2011.tif
File cropscape-strawberries-06053-2011.tif uploaded to agrisense/cropscape_monterey/cropscape-strawberries-06053-2011.tif
Local file cropscape-strawberries-06053-2011.tif deleted.
Pulling Strawberry Data for year 2012 and 06053 FIPS
File downloaded to cropscape-strawberries-06053-2012.tif
File cropscape-strawberries-06053-2012.tif uploaded to agrisense/cropscape_monterey/cropscape-strawberries-06053-2012.tif
Local file cropscape-strawberries-06053-2012.tif deleted.
Pulling Strawberry Data for year 2013 and 06053 FIPS
File downloaded to cropscape-strawberries-06053-2013.tif
File cropscape-straw